In [4]:
from glob import glob
import os, pandas as pd
import cv2, numpy as np
import matplotlib.pyplot as plt
from segmentation_models import *
from segmentation_models.metrics import iou_score
import shutil
import pickle

/tmp/ipykernel_400532/1286792484.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import os, pandas as pd
2024-02-01 01:01:23.595605: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-01 01:01:23.752773: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one 

Segmentation Models: using `keras` framework.


In [5]:
df = pd.read_csv('/home/lucas_araujo/pibic-2024/dataset/Data_Entry_2017.csv')
df = df.loc[:,['Image Index','Patient ID', 'Finding Labels']]
img_paths={os.path.basename(x): x for x in glob(os.path.join('.', '/home/lucas_araujo/pibic-2024/dataset', 'images*','images','*.png'))} 
df['path']=df['Image Index'].map(img_paths.get) #mapping image ids to all image paths
df

,Image Index,Patient ID,Finding Labels,path
0,00000001_000.png,1,Cardiomegaly,/home/lucas_araujo/pibic-2024/dataset/images_0...
1,00000001_001.png,1,Cardiomegaly|Emphysema,/home/lucas_araujo/pibic-2024/dataset/images_0...
2,00000001_002.png,1,Cardiomegaly|Effusion,/home/lucas_araujo/pibic-2024/dataset/images_0...
3,00000002_000.png,2,No Finding,/home/lucas_araujo/pibic-2024/dataset/images_0...
4,00000003_000.png,3,Hernia,/home/lucas_araujo/pibic-2024/dataset/images_0...
...,...,...,...,...
112115,00030801_001.png,30801,Mass|Pneumonia,/home/lucas_araujo/pibic-2024/dataset/images_0...
112116,00030802_000.png,30802,No Finding,/home/lucas_araujo/pibic-2024/dataset/images_0...
112117,00030803_000.png,30803,No Finding,/home/lucas_araujo/pibic-2024/dataset/images_0...
112118,00030804_000.png,30804,No Finding,/home/lucas_araujo/pibic-2024/dataset/images_0...


In [6]:
def save_masks(pred, names, output):
    """
        Salva as máscaras geradas

        Parameters
        ----------
        pred: array-like, float32
            Array contendo todas as máscaras preditas
        names: list, 'string'
            Lista de nomes dos arquivos das imagens
        output: string
            Nome do diretório onde as imagens serão salvas
            
    """
    for i in range(pred.shape[0]):
        m = pred[i, :, :, 0]
        cv2.imwrite(output + names[i] + '.png', 255 * ((m - m.min())/(m.max() - m.min())))

In [7]:
batch = []
names = []
i = 1
BACKBONE = 'densenet169'
name = 'Unet'
# model = Unet(BACKBONE, input_shape=(512, 512, 3), classes=1)
# model.load_weights('/home/lucas_araujo/pibic-2024/classificacao_multilabel_chest/segm_model/segm_models/Unet/weights/densenet169_best_weights.hdf5')
pre_processing = get_preprocessing(BACKBONE)

os.makedirs("./mask_chest", exist_ok=True)

output_dir_base = 'mask_chest/images_'  # Diretório base para salvar as imagens
output_dir_counter = 0

for (path, name) in zip(df['path'], df['Image Index']):
    img = cv2.imread(path)
    img = cv2.resize(img, (512, 512))
    batch.append(img)
    names.append(name)
    print(f"{i}/1000")
    if i % 1000 == 0:
        output_dir = f'{output_dir_base}{output_dir_counter}/'
        os.makedirs(output_dir, exist_ok=True)

        batch = np.array(batch)
        model = Unet(BACKBONE, input_shape=(512, 512, 3), classes=1)
        model.load_weights('/home/lucas_araujo/pibic-2024/classificacao_multilabel_chest/segm_model/segm_models/Unet/weights/densenet169_best_weights.hdf5')
        X = pre_processing(batch)
        pred = model.predict(X, verbose=1)

        # Salvar masks dentro da pasta específica
        save_masks(pred, names, output_dir)

        # Limpar os batches e nomes
        batch = []
        names.clear()

        # Incrementar o contador de diretório
        output_dir_counter += 1

    i += 1

# Processar os dados restantes
if batch:
    output_dir = f'{output_dir_base}{output_dir_counter}/'
    os.makedirs(output_dir, exist_ok=True)

    batch = np.array(batch)
    X = pre_processing(batch)
    pred = model.predict(X, verbose=1)

    # Salvar masks dentro da pasta específica
    save_masks(pred, names, output_dir)

2024-02-01 01:01:34.907206: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-01 01:01:34.907890: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-01 01:01:34.938276: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-01 01:01:34.939758: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-01 01:01:34.940962: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000


libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
124/1000
125/1000
126/1000
127/1000
128/1000
129/1000
130/1000
131/1000
132/1000
133/1000
134/1000
135/1000
136/1000
137/1000
138/1000
139/1000
140/1000
141/100

2024-02-01 01:01:51.245793: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3145728000 exceeds 10% of free system memory.
2024-02-01 01:01:53.216939: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3145728000 exceeds 10% of free system memory.
2024-02-01 01:01:56.731550: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8700
2024-02-01 01:01:57.117223: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-01 01:01:58.890953: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.99GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


...

32/32 [==============================] - 20s 453ms/step
1001/1000
1002/1000
1003/1000
1004/1000
1005/1000
1006/1000
1007/1000
1008/1000
1009/1000
1010/1000
1011/1000
1012/1000
1013/1000
1014/1000
1015/1000
1016/1000
1017/1000
1018/1000
1019/1000
1020/1000
1021/1000
1022/1000
1023/1000
1024/1000
1025/1000
1026/1000
1027/1000
1028/1000
1029/1000
1030/1000
1031/1000
1032/1000
1033/1000
1034/1000
1035/1000
1036/1000
1037/1000
1038/1000
1039/1000
1040/1000
1041/1000
1042/1000
1043/1000
1044/1000
1045/1000
1046/1000
1047/1000
1048/1000
1049/1000
1050/1000
1051/1000
1052/1000
1053/1000
1054/1000
1055/1000
1056/1000
1057/1000
1058/1000
1059/1000
1060/1000
1061/1000
1062/1000
1063/1000
1064/1000
1065/1000
1066/1000
1067/1000
1068/1000
1069/1000
1070/1000
1071/1000
1072/1000
1073/1000
1074/1000
1075/1000
1076/1000
1077/1000
1078/1000
1079/1000
1080/1000
1081/1000
1082/1000
1083/1000
1084/1000
1085/1000
1086/1000
1087/1000
1088/1000
1089/1000
1090/1000
1091/1000
1092/1000
1093/1000
1094/1000
1095

2024-02-01 01:02:31.217769: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3145728000 exceeds 10% of free system memory.
2024-02-01 01:02:33.104420: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3145728000 exceeds 10% of free system memory.


32/32 [==============================] - 13s 420ms/step
2001/1000
2002/1000
2003/1000
2004/1000
2005/1000
2006/1000
2007/1000
2008/1000
2009/1000
2010/1000
2011/1000
2012/1000
2013/1000
2014/1000
2015/1000
2016/1000
2017/1000
2018/1000
2019/1000
2020/1000
2021/1000
2022/1000
2023/1000
2024/1000
2025/1000
2026/1000
2027/1000
2028/1000
2029/1000
2030/1000
2031/1000
2032/1000
2033/1000
2034/1000
2035/1000
2036/1000
2037/1000
2038/1000
2039/1000
2040/1000
2041/1000
2042/1000
2043/1000
2044/1000
2045/1000
2046/1000
2047/1000
2048/1000
2049/1000
2050/1000
2051/1000
2052/1000
2053/1000
2054/1000
2055/1000
2056/1000
2057/1000
2058/1000
2059/1000
2060/1000
2061/1000
2062/1000
2063/1000
2064/1000
2065/1000
2066/1000
2067/1000
2068/1000
2069/1000
2070/1000
2071/1000
2072/1000
2073/1000
2074/1000
2075/1000
2076/1000
2077/1000
2078/1000
2079/1000
2080/1000
2081/1000
2082/1000
2083/1000
2084/1000
2085/1000
2086/1000
2087/1000
2088/1000
2089/1000
2090/1000
2091/1000
2092/1000
2093/1000
2094/1000
2095

2024-02-01 01:03:03.574084: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3145728000 exceeds 10% of free system memory.
2024-02-01 01:03:16.870491: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 512.00MiB (rounded to 536870912)requested by op model/conv1/bn/FusedBatchNormV3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-02-01 01:03:16.870529: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2024-02-01 01:03:16.870537: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 154, Chunks in use: 153. 38.5KiB allocated for chunks. 38.2KiB in use in bin. 6.0KiB client-requested in use in bin.
2024-02-01 01:03:16.870543: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin

ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv1/bn/FusedBatchNormV3' defined at (most recent call last):
    File "/home/lucas_araujo/.conda/envs/lucas_ag/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/lucas_araujo/.conda/envs/lucas_ag/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/lucas_araujo/.conda/envs/lucas_ag/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/lucas_araujo/.conda/envs/lucas_ag/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/lucas_araujo/.conda/envs/lucas_ag/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/lucas_araujo/.conda/envs/lucas_ag/lib/python3.10/site-packages/tornado/platform/asyncio.py", lin

In [2]:
import os
os.listdir('./mask_chest/')

[]